# Extent of Occurrence (EOO) and Area of Occupancy (AOO)

## Table of contents
1. [Setting up](#setup)
2. [Loading R packages](#packages)
3. [Loading data](#data)
4. [Calculate Extent of Occurrence](#eoo)
5. [Calculate Area of Occupancy](#aoo)
6. [IUCN evaluation](#iucn)

## Setting up <a name="setup"></a>

If you opened this notebook from the GitHub *ecocloud* repository, please save it first to your own workspace to ensure that any of your changes will be saved. To create a new folder and save this notebook to your workspace follow these steps:
1. Click on the 'folder' icon in the menubar on the left (top icon).
2. Double click on the workspace folder.
3. Click on the 'new folder' icon in the menubar on the top. This will create a new Untitled Folder.
4. Right click on the Untitled Folder, click Rename, and name it appropriately.
5. Double click on the folder name to move into the folder.
6. Go to 'File' > 'Save Notebook'.
7. It is really important that when you save your notebook, you save it in the following format: "workspace/Folder Name/my-notebook.ipynb". Without the "workspace/" in the name the notebook will not be saved in the workspace for future use. 

Set your working directory to the folder you just created, by executing the code in the cell below. NB. Change the text *biodiversity_next* to the name of your folder.

In [ ]:
setwd("/home/jovyan/workspace/biodiversity_next")
getwd()

## Loading R packages <a name="packages"></a>

*ecocloud* has pre-installed many R packages that are often used by ecology researchers, but you can install packages yourself if needed (use install.packages command). The cell below installs and loads the libraries needed to execute the script in this notebook.

In [ ]:
install.packages("ConR")
library(ConR)
library(sp)
library(maptools)

## Loading data <a name="data"></a>

To calculate the EOO and AOO of a species, you need a species occurrence file with latitude and longitude records. See the GBIF-data notebook how to get species data from GBIF. To use the functions in this notebook, the data need to be organised in a dataframe with the columns: latitude, longitude, species, in this exact order.

You can either use your own species file if it is correctly structured, or use the example file: ochotona-princeps.csv that you can find in the *ecocloud* Github repository.

Adapt the following code to read in a csv file (change filename to the actual name of the file):

`spp.data <- read.csv("filename.csv")`

`head(spp.data)` to check if the dataframe has three columns in the correct order: latitude, longitude, species

To be able to plot the records later on the map, you can adapt the code below (if needed) to create a SpatialPointsDataFrame:

In [ ]:
species_points <- SpatialPointsDataFrame(
  coords=cbind(spp.data$longitude, spp.data$latitude),
  data=spp.data,
  proj4string=CRS('+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0')
)

## Calculate Extent of Occurrence (EOO) <a name="eoo"></a>

The extent of occurrence (EOO) is defined as the area contained within the shortest continuous imaginary boundary which can be drawn to encompass all the known occurrence records. This area can often be measured by a minimum convex polygon. The EOO does not account for discontinuities or disjunctions within the overall distributions of the species such as large areas of obviously unsuitable habitat.

This example uses the [EOO.computing function](https://www.rdocumentation.org/packages/ConR/versions/1.0/topics/EOO.computing) in the ConR package.

Adapt and execute the code below to calculate EOO. This also creates an EOO_results.csv file. You can also generate a shapefile of the EOO by setting `write_shp=TRUE`. In that case, a new folder called 'shapesIUCN' will be created.

In [ ]:
EOO <- EOO.computing(spp.data, file.name="EOO_results", country_map=land, export_shp=TRUE, write_shp=FALSE)
print(EOO)

Execute the code below to plot the occurrence records and the convex hull that represents the EOO:

In [ ]:
plot(EOO$spatial.polygon_1, col="grey")
plot(land, add=T)
plot(species_points,add=TRUE)

## Calculate Area of Occupancy (AOO) <a name="eoo"></a>

The Area of Occupancy (AOO) is defined as the area within its ‘extent of occurrence’ which is occupied by a species, excluding cases of vagrancy. The measure reflects the fact that a taxon will not usually occur throughout the area of its extent of occurrence, which may contain unsuitable or unoccupied habitats. In some cases (e.g. irreplaceable colonial nesting sites, crucial feeding sites for migratory taxa) the area of occupancy is the smallest area essential at any stage to the survival of existing populations of a taxon. 

The size of the area of occupancy is a function of the scale at which it is measured. The recommendation by the International Union for Conservation of Nature (IUCN) for the grid size used to calculate AOO is 2km.

This example uses the [AOO.computing function](https://www.rdocumentation.org/packages/ConR/versions/1.2.4/topics/AOO.computing) in the ConR package.

Adapt and execute the code below to calculate AOO:

In [ ]:
AOO <- AOO.computing(spp.data, Cell_size_AOO = 2)
print(AOO)

## IUCN evaluation <a name="iucn"></a>

The ConR package ([Dauby et al. 2017](https://onlinelibrary.wiley.com/doi/full/10.1002/ece3.3704)) includes a function (IUCN.eval) that provides values for the EOO and AOO as well as an estimate of the number of subpopulations and locations. Based on these parameters the function gives in an indication in which IUCN category the species falls with regards to criterion B. This criterion has  been  designed  to  identify  populations  with  restricted  distributions  that  are also  severely  fragmented.

The function can be executed with the code in the cell below. Other optional parameters can be found in the [documentation](https://www.rdocumentation.org/packages/ConR/versions/1.2.4/topics/IUCN.eval).

After the code has run, a new file 'IUCN_results' is created as well as a folder 'IUCN_results_map'. This folder includes a png file with a map of the occurrences and the EOO as well as the values for all parameters.

In [ ]:
IUCN.eval(spp.data, DrawMap = TRUE, add.legend = TRUE)

Have a look at the IUCN status for your species.